In [2]:
import numpy as np
import rnn_utils

In [3]:
 # 一个时间步的前向
def rnn_cell_forward(xt, a_prev, parameters):
    """
    多样本单步传播
    :param xt: 输入矩阵（每一列表示表示一个单词）
    :param a_prev: 记忆值
    :param parameters: 权重和偏置参数
    :return: 生成的记忆、预测，缓存（两个记忆、输入、parameters）
    """

    waa = parameters['waa']
    wax = parameters['wax']
    ba  = parameters['ba']
    wya = parameters['wya']
    by  = parameters['by']

    a_next = np.tanh(np.dot(waa, a_prev) + np.dot(wax, xt) + ba)
    y_pred = rnn_utils.softmax(np.dot(wya, a_next) + by)

    cache = (a_next, a_prev, xt, parameters)

    return a_next, y_pred, cache

In [5]:
# 测试函数模块
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"waa": Waa, "wax": Wax, "wya": Wya, "ba": ba, "by": by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next = ", a_next)
print("a_next.shape = ", a_next.shape)
print("yt_pred=", yt_pred)
print("yt_pred.shape = ", yt_pred.shape)

a_next =  [[ 0.95442347 -0.97959841 -0.77682357 -0.85960496  0.2996283  -0.72828789
   0.70341981  0.396781    0.64215271 -0.68720152]
 [-0.77817006 -0.96939535 -0.90158668 -0.89269334 -0.94794605 -0.62569074
  -0.7847199   0.73807292  0.40638533 -0.49874722]
 [ 0.34337788 -0.99997631 -0.99692205 -0.98133709 -0.93123291 -0.99802557
  -0.99662894 -0.93641136 -0.25153222  0.54770565]
 [-0.85404662  0.97190276  0.60516394  0.65999969 -0.68038654  0.09222782
   0.34729991  0.41705046 -0.44431726  0.74395075]
 [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
  -0.18887155  0.99815551  0.6531151   0.82872037]]
a_next.shape =  (5, 10)
yt_pred= [[0.0111839  0.98317979 0.78859101 0.63182533 0.01011613 0.11054788
  0.63079776 0.0033688  0.0017441  0.82253474]
 [0.9888161  0.01682021 0.21140899 0.36817467 0.98988387 0.88945212
  0.36920224 0.9966312  0.9982559  0.17746526]]
yt_pred.shape =  (2, 10)


In [11]:
# 整个时间序列
def rnn_forward(x, a0, parameters):
    """
    根据图3来实现循环神经网络的前向传播
    
    参数：
        x -- 输入的全部数据，维度为(n_x, m, T_x)
        a0 -- 初始化隐藏状态，维度为 (n_a, m)
        parameters -- 字典，包含了以下内容:
                        Wax -- 矩阵，输入乘以权重，维度为（n_a, n_x）
                        Waa -- 矩阵，隐藏状态乘以权重，维度为（n_a, n_a）
                        Wya -- 矩阵，隐藏状态与输出相关的权重矩阵，维度为（n_y, n_a）
                        ba  -- 偏置，维度为（n_a, 1）
                        by  -- 偏置，隐藏状态与输出相关的偏置，维度为（n_y, 1）
    
    返回：
        a -- 所有时间步的隐藏状态，维度为(n_a, m, T_x)
        y_pred -- 所有时间步的预测，维度为(n_y, m, T_x)
        caches -- 为反向传播的保存的元组，维度为（【列表类型】cache, x)）
    """
    
    # 初始化“caches”，它将以列表类型包含所有的cache
    caches = []
    
    # 获取 x 与 Wya 的维度信息
    n_x, m, T_x = x.shape
    n_y, n_a = parameters['wya'].shape
    
    # 使用0来初始化“a” 与“y”
    a = np.zeros([n_a, m, T_x])
    y_pred = np.zeros([n_y, m, T_x])
    
    # 初始化“next”
    a_next = a0
    
    # 遍历所有时间步
    for t in range(T_x):
        ## 1.使用rnn_cell_forward函数来更新“next”隐藏状态与cache。
        a_next, yt_pred, cache = rnn_cell_forward(x[:, :, t], a_next, parameters)
        
        ## 2.使用 a 来保存“next”隐藏状态（第 t ）个位置。
        a[:, :, t] = a_next
        
        ## 3.使用 y 来保存预测值。
        y_pred[:, :, t] = yt_pred
        
        ## 4.把cache保存到“caches”列表中。
        caches.append(cache)
    
    # 保存反向传播所需要的参数
    caches = (caches, x)
    
    return a, y_pred, caches


In [16]:
# 测试
np.random.seed(1)
x = np.random.randn(3,2,4)  # 输入层维度为3， 2个样本， 每个样本4个单词
a0 = np.random.randn(5,2)  # 时间片矩阵信息
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"waa": Waa, "wax": Wax, "wya": Wya, "ba": ba, "by": by}

a, y_pred, caches = rnn_forward(x, a0, parameters)
print("a[4][1] = ", a[4][1])  # 第四个隐层单元对应的第一个样本的四个时间片记忆【隐层单元 样本 时间片】
print("a.shape = ", a.shape)
# print("y_pred[1][3] =", y_pred[1][3])
# print("y_pred.shape = ", y_pred.shape)
# print("caches[1][1][3] =", caches[1][1][3])
# print("len(caches) = ", len(caches))


a[4][1] =  [-0.95646207  0.93390825 -0.2461967   0.98521248]
a.shape =  (5, 2, 4)
